## Setting up the development environment

In [1]:
!pip install cohere tiktoken openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00


## Loading data

In [2]:
!pip install python-dotenv

In [3]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

file_name = "customer_review.csv"

df = pd.read_csv(file_name)
df.shape

(100, 2)

In [5]:
df.sample(3)

,product_name,customer_review
95,PlushPout Hydrating Lip Balm in Tropical Punch,"Apart from the delicious tropical scent, this balm keeps my lips moisturized for hours. The subtle tint it imparts is an added bonus, making it perfect for those no-makeup days."
61,LushLips Matte Lipstick in Cherry Bliss,"The formula glides on smoothly. It's not transfer-proof, but the shade is so stunning I don't mind reapplying."
51,LushLips Matte Lipstick in Cherry Bliss,"Although the shade is to die for, I wish it wasn't so drying. It's best used with a lip primer or balm."


In [6]:
product_review = df.iloc[50]['customer_review']
print(product_review)

This highlighter is subtle and perfect for everyday use. I just wish it had a more intense shine for special occasions.


In [7]:
prompt_template = """
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer using one of the following words: happy, neutral, dissatisfied
Be concise.

Review text: ```{product_review}```
"""

In [8]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm_name = "gpt-4"

llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

In [9]:
ans = llm_chain.predict(product_review=product_review)
print(ans)

neutral


## Running over the table

In [10]:
df_sub = df.loc[20:40]
results = []

# Iterate over each row
for index, row in df_sub.iterrows():
    # Apply LLM chain on the desired column in each row
    result = llm_chain.predict(product_review = row['product_name']+' '+row['customer_review'])

    results.append(result)

In [11]:
results

['dissatisfied',
 'dissatisfied',
 'dissatisfied',
 'dissatisfied',
 'dissatisfied',
 'happy',
 'dissatisfied',
 'dissatisfied',
 'dissatisfied',
 'happy',
 'neutral',
 'dissatisfied',
 'dissatisfied',
 'neutral',
 'neutral',
 'dissatisfied',
 'neutral',
 'neutral',
 'neutral',
 'happy',
 'neutral']

In [12]:
df_sub['sentiment'] = results
df_sub

<ipython-input-12-66e4f84c77d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['sentiment'] = results


,product_name,customer_review,sentiment
20,DreamyDew Hydration Mist with Lavender,The scent is overpowering and gives me a headache. Doesn't feel hydrating either.,dissatisfied
21,LuxLash Volumizing Mascara in Charcoal,It's just okay. Gives volume but dries out quickly and flakes by afternoon.,dissatisfied
22,SilkyMatte Liquid Lip in Coral Dreams,"Loved the color, but it's not transfer-proof. Left marks on my coffee cup.",dissatisfied
23,RefreshMe Under-eye Brightening Cream,"It did reduce puffiness, but no impact on my dark circles. Still searching for the perfect product.",dissatisfied
24,SeamlessCover Full Coverage Concealer,It settled into my fine lines and looked patchy after a few hours. Not impressed.,dissatisfied
25,GoldenHour Bronzer Stick,Easy to apply and blends seamlessly. Perfect for that sun-kissed look!,happy
26,PoreVanish Face Primer,It felt greasy on my skin and didn't hold up my foundation well. I'll pass.,dissatisfied
27,EternalGlow Illuminating Eye Serum,"Lightweight and absorbs quickly. However, the results are not noticeable.",dissatisfied
28,VelvetFinish Blush Palette in Sunset Hues,Colors are beautiful but not long-lasting. They faded within a few hours.,dissatisfied
29,BerryBurst Antioxidant Face Mask,"It tingles a bit, but my skin looked brighter after use. Smells delicious too!",happy
